In [3]:
#引入库
import pandas as pd
import torch
from torch import nn, optim
from torchvision.datasets import mnist
from torchvision.transforms import ToTensor
import time

In [4]:
#建立lenet模型
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 5), #输入通道， 输出通道，大小
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5), #输入通道， 输出通道，大小
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.fc = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),
        )
    def forward(self, train_x):
        y = self.conv(train_x)
        y = self.fc(y.view(y.shape[0],-1))
        return y

In [5]:
#训练函数
def train(net, num_epochs, learning_rate, batch_size):
    
    optimizer = optim.Adam(params=net.parameters(), lr=learning_rate)
    
    train_dataset = mnist.MNIST(root='./mnist/train', train=True, transform=ToTensor())
    test_dataset = mnist.MNIST(root='./mnist/test', train=False, transform=ToTensor())
    train_iter = torch.utils.data.DataLoader(train_dataset, batch_size)
    test_iter = torch.utils.data.DataLoader(test_dataset, batch_size)
    
    for epoch in range(num_epochs):
        start = time.time()
        train_acc, train_loss, test_acc = 0.0, 0.0, 0.0
        for x, y in train_iter:
            y_hat = net(x)
            l = loss(y_hat, y).mean()
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            
            train_loss = l.item()
            train_acc = (y_hat.argmax(dim=1) == y).float().mean().item()
        
        for x, y in test_iter:
            test_acc = (net(x).argmax(dim=1) == y).float().mean().item()
            
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1, train_loss, train_acc, test_acc, time.time() - start))
#训练与测试
net = LeNet()
loss = nn.CrossEntropyLoss()
batch_size = 64
num_epochs = 3
learning_rate = 1e-2
weight_decay = 0

train(net, num_epochs, learning_rate, batch_size)

epoch 1, loss 0.0118, train acc 1.000, test acc 1.000, time 29.7 sec
epoch 2, loss 0.0043, train acc 1.000, test acc 1.000, time 28.7 sec
epoch 3, loss 0.2816, train acc 0.969, test acc 0.938, time 27.3 sec


In [6]:
#模型效果展示
